# Introduction to Monte Carlo Simulation

This notebook provides a brief introduction to Monte Carlo simulation and demonstrates the Monte Carlo toolkit.

## What is Monte Carlo Simulation?

Monte Carlo simulation is a computational technique that uses random sampling to estimate the behavior of complex systems or compute numerical results. The method is named after the Monte Carlo Casino in Monaco, reflecting the use of randomness and probability.

**Key principles:**
1. **Law of Large Numbers**: As the number of trials increases, the sample mean converges to the true expected value.
2. **Standard Error**: The uncertainty in our estimate decreases as \(SE \approx \frac{s}{\sqrt{N}}\), where \(s\) is the sample standard deviation and \(N\) is the number of trials.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from montecarlo.core import Simulation
from montecarlo.distributions import normal, lognormal

# Set style for cleaner plots
plt.style.use('default')
plt.rcParams['figure.figsize'] = (8, 6)

print("Monte Carlo Toolkit loaded successfully!")


## Example 1: Estimating π

One of the classic Monte Carlo examples is estimating π by sampling random points in a unit square and counting how many fall inside a unit circle.

The estimate is: \(\pi \approx 4 \times \frac{\text{points inside circle}}{\text{total points}}\)


In [ ]:
# Define model: check if point is inside unit circle
def model(x):
    x_coords = x[:, 0]
    y_coords = x[:, 1]
    inside = (x_coords**2 + y_coords**2) <= 1.0
    return inside.astype(float)

# Define sampler: uniform points in [0,1] x [0,1]
def sampler(rng, size):
    return rng.uniform(0, 1, size=(size, 2))

# Run simulation
sim = Simulation(model, sampler, seed=42)
result = sim.run(1_000_000, keep=True)

pi_estimate = 4 * result["mean"]
se_pi = 4 * result["se"]

print(f"π estimate: {pi_estimate:.6f}")
print(f"Standard error: {se_pi:.6f}")
print(f"True π: {np.pi:.6f}")
print(f"Error: {abs(pi_estimate - np.pi):.6f}")
print(f"Trials: {result['n']:,}")


### Convergence Analysis

Let's examine how the estimate converges as we increase the number of trials.


In [ ]:
# Run multiple simulations with increasing N
n_values = [1000, 5000, 10000, 50000, 100000, 500000, 1000000]
pi_estimates = []

for n in n_values:
    sim_n = Simulation(model, sampler, seed=42)
    result_n = sim_n.run(n)
    pi_est = 4 * result_n["mean"]
    pi_estimates.append(pi_est)

# Plot convergence
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(n_values, pi_estimates, 'o-', label='Estimate', linewidth=2, markersize=8)
ax.axhline(y=np.pi, color='r', linestyle='--', label=f'True π = {np.pi:.6f}', linewidth=2)
ax.set_xlabel('Number of Trials', fontsize=12)
ax.set_ylabel('π Estimate', fontsize=12)
ax.set_title('Convergence of π Estimate', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xscale('log')
plt.tight_layout()
plt.show()

print("\nNote: As N increases, the estimate converges to the true value.")
print(f"Standard error decreases as ~1/√N: SE ≈ s/√N")


## Example 2: Linear Model

Now let's simulate a linear model: \(y = x_1 + 2x_2\), where:
- \(x_1 \sim N(\mu_1, \sigma_1)\)
- \(x_2 \sim \text{LogNormal}(\mu_2, \sigma_2)\)


In [ ]:
# Define model
def linear_model(x):
    return x[:, 0] + 2 * x[:, 1]

# Define sampler
def linear_sampler(rng, size):
    x1 = normal(10.0, 2.0)(rng, size)
    x2 = lognormal(1.0, 0.25)(rng, size)
    return np.column_stack([x1, x2])

# Run simulation
sim_linear = Simulation(linear_model, linear_sampler, seed=42)
result_linear = sim_linear.run(200_000, keep=True)

print("Linear Model Results:")
print(f"  Mean: {result_linear['mean']:.4f}")
print(f"  Std Dev: {result_linear['std']:.4f}")
print(f"  5th percentile: {result_linear['p05']:.4f}")
print(f"  Median: {result_linear['p50']:.4f}")
print(f"  95th percentile: {result_linear['p95']:.4f}")
print(f"  Standard error: {result_linear['se']:.6f}")


### Distribution Visualization

Let's visualize the output distribution with a histogram and annotate key statistics.


In [ ]:
y = result_linear["all"]

fig, ax = plt.subplots(figsize=(10, 6))
n, bins, patches = ax.hist(y, bins=60, density=True, alpha=0.7, color='steelblue', edgecolor='black', linewidth=0.5)

# Annotate mean and percentiles
mean_val = result_linear['mean']
p05_val = result_linear['p05']
p95_val = result_linear['p95']

ax.axvline(mean_val, color='green', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')
ax.axvline(p05_val, color='orange', linestyle='--', linewidth=2, label=f'5th %ile: {p05_val:.2f}')
ax.axvline(p95_val, color='red', linestyle='--', linewidth=2, label=f'95th %ile: {p95_val:.2f}')

ax.set_xlabel('Output Value (y)', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title('Distribution of y = x₁ + 2x₂', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nThe distribution shows the uncertainty in y given the uncertainty in x₁ and x₂.")
print(f"95% of outcomes fall between {p05_val:.2f} and {p95_val:.2f}.")


## Summary

Monte Carlo simulation is a powerful tool for:
- **Propagating uncertainty** through complex models
- **Estimating probabilities** of rare events
- **Computing risk metrics** (VaR, CVaR, etc.)
- **Sensitivity analysis** of model inputs

**Key takeaways:**
1. More trials → better accuracy (SE ∝ 1/√N)
2. Vectorized operations are essential for performance
3. Fixed seeds ensure reproducibility
4. Always validate input/output shapes in your models
